<center>
<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="150">
</p>



<h1>Curso Procesamiento de Lenguaje Natural</h1>

<h3>RNN a pie</h3>


<p> Julio Waissman Vilanova </p>
<p>
<img src="https://identidadbuho.unison.mx/wp-content/uploads/2019/06/letragrama-cmyk-72.jpg" width="150">
</p>


<a target="_blank" href="https://colab.research.google.com/github/mcd-unison/pln/blob/main/labs/RNN/Estados-ocultos.ipynb"><img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;"  width="30" /> Ejecuta en Colab</a>

<p>
Tomado parcialmente y adaptado de libretas de la <i>Especialización en procesamiento de lenguaje natural</i> de <i>Deeplearning.ai</i>, disponible en <i>Coursera</i>.
</p>


</center>


## RNN vainilla

Para una RNN simple como se muestra en la figura:

![vanilla rnn](https://github.com/mcd-unison/pln/blob/main/labs/RNN/vanilla_rnn.PNG?raw=1)


La activación de los estados ocultos están dados por:      

$h^{<t>}=g(W_{hh}h^{<t-1>} + W_{hx}x^{<t>} + b_h)$                                        


este ejemplo lo vams a hacer usando exclusivamente `numpy` para entender el modelo.

In [1]:
import numpy as np
from time import perf_counter

Vamos entonces a desarrollar la función de alimentación a adelante de una RNN

In [2]:
def sigmoid(x):
    # Calcula la función logística
    return 1 / (1 + np.exp(-x))
    ## ACABA CODIGO

In [3]:
def forward_V_RNN(inputs, weights):
    # Forward propagation para una RNN vanilla
    x_t, h_t_prev = inputs

    # weights.
    w_hh, w_xh, b_h = weights

    ### INICIA CODIGO ###
    # Nuevo estado oculto

    # Operación lineal
    z_t = np.dot(w_xh, x_t) + np.dot(w_hh, h_t_prev) + b_h

    # Activación
    h_t = sigmoid(z_t)

    ### ACABA CODIGO ###

    return h_t

Vamos a probar como funciona

In [4]:
# Data

nh = 2   # Dimensión del vector de variables ocultas
nx = 3   # Dimensión del vector de entrada

w_hh = np.full((nh, nh), 1.)  # 3x2 llenado con puros 1s
w_hx = np.full((nh, nx), 9.)  # 3x3 llenado con puros 9s
h_t_prev = np.full((nh, 1), 1.)  # 2x1 llenado con puros 1s
x_t = np.full((nx, 1), 9.)       # 3x1 llenado con puros 9s
b_h = np.zeros((nh, 1))

# Si prefieres valores aleatorios, descomenta lo siguiente:

# w_hh = np.random.standard_normal((nh,nh))
# w_hx = np.random.standard_normal((nh,nx))
# h_t_prev = np.random.standard_normal((nh,1))
# x_t = np.random.standard_normal((nx,1))

# Aplicando un solo paso
h_t = forward_V_RNN([x_t, h_t_prev], [w_hh, w_hx, b_h])

print("\nValor h_t:")
print(h_t, "\n")


Valor h_t:
[[1.]
 [1.]] 



## RNN tipo LSTM

Una LST es un modelo como el que se muestra en la figura, con todo y sus ecuaciones

![](https://github.com/mcd-unison/pln/blob/main/labs/RNN/LSTM.jpg?raw=1)

Como podemos ver tenemos 3 vectores de entrada a la celda:

- $h^{<t-1>}$ el vector de variables ocultas provenientes de un paso anterior,
- $C^{<t-1>}$ el vector de valores de celda (memoria de largo plazo) provenientes de un paso anterior,
- $x^{<t>}$ el vector de variables de entrada. Idealmente debería estar normalizado entre -1 y 1 cada uno de los valores de entrada.

Como podemos ver tenemos varias operaciones:

- Una compuerta de olvido $f$ que depende de $h^{<t-1>}$ y $x^{<t>}$ cuya salida es un vector del tamaño de las variables ocultas con valores entre 0 y 1 con la importancia que debe tener el valor de celda anterior (memoria de largo plazo)

- Una compuerta de entrada $i$ que depende de $h^{<t-1>}$ y $x^{<t>}$ cuya salida es un vector del tamaño de las variables ocultas con valores entre 0 y 1 con la importancia que debe tener la activación de la celda actual (memoria de corto plazo)

- Una compuerta de salida $i$ que depende de $h^{<t-1>}$ y $x^{<t>}$ cuya salida es un vector del tamaño de las variables ocultas con valores entre 0 y 1 con la importancia que debe tener el valor de celda actual en el valor de la de la variable oculta correspondiente.

- El calculo de la activación actual, que depende de $h^{<t-1>}$ y $x^{<t>}$, el cual se hace con una tangente hiperbólica, para mantener los valores entre -1 y 1.


Hagamos entonces una celda LSTM


In [5]:
def forward_LSTM(inputs, weights):
    # Forward propagation para una RNN tipo LSTM
    x_t, h_t_prev, C_t_prev = inputs

    # weights.
    Ui, Wi, Uf, Wf, Uo, Wo, U, W = weights

    ### INICIA CODIGO ###
    # Nuevo estado oculto y valor de celda

    # Compuerta de entrada
    i = sigmoid(np.dot(Ui, x_t) + np.dot(Wi, h_t_prev))

    # Compuerta de olvido
    f = sigmoid(np.dot(Uf, x_t) + np.dot(Wf, h_t_prev))

    # Compuerta de salida
    o = sigmoid(np.dot(Uo, x_t) + np.dot(Wo, h_t_prev))

    # Valor de celda de memoria de corto plazo
    C_t_short = np.tanh(np.dot(U, x_t) + np.dot(W, h_t_prev))

    # Valor de celda de memoria de corto y largo plazo
    C_t = f * C_t_prev + i * C_t_short

    # Valor de variable oculta
    h_t = o * np.tanh(C_t)

    ### ACABA CODIGO ###

    return h_t, C_t

Vamos a probar como funciona

In [6]:
# Data

nh = 2   # Dimensión del vector de variables ocultas
nx = 3   # Dimensión del vector de entrada

Ui = np.random.standard_normal((nh,nx))
Wi = np.random.standard_normal((nh,nh))

Uf = np.random.standard_normal((nh,nx))
Wf = np.random.standard_normal((nh,nh))

Uo = np.random.standard_normal((nh,nx))
Wo = np.random.standard_normal((nh,nh))

U = np.random.standard_normal((nh,nx))
W = np.random.standard_normal((nh,nh))


h_t_prev = 2 * np.random.standard_normal((nh,1)) - 1
C_t_prev = np.random.standard_normal((nh,1))
x_t = 2 * np.random.standard_normal((nx,1)) - 1

# Aplicando un solo paso
h_t, C_t = forward_LSTM(
    [x_t, h_t_prev, C_t_prev],
    [Ui, Wi, Uf, Wf, Uo, Wo, U, W]
)

print("\nValor h_t:")
print(h_t, "\n")

print("\nValor C_t:")
print(C_t, "\n")



Valor h_t:
[[0.00832841]
 [0.69462367]] 


Valor C_t:
[[0.00876102]
 [0.92568422]] 



## La función `scan`para el cálculo de BPTT

La función `scan` se usa para calcular la propagación hacia adelante. Si la funcións e implementa en un *framework* como *Tensorflow* o *pytorch*, entonces se puede ir guardando los gradientes de cada aplicación a lo largo del tiempo y usarlos en el calculo del gradiente para la función de aprendizaje.

Aquí solo vamos a tratar de mostrar como funcionaría dicha función, la cual recibe:

- `elems` : lista de entradas (`X`)
- `weights` : los parámetros que necesita la función de feedforward para su cálculo (pesos)
- `h_0` : estado oculto inicial

`scan` va por todos los valores de `x` en `elems`, llama la función de feedforward con los argumentos necesarios, guarda el estado oculto `h_t` y agrega el valor de `h_t` a una lista.

Vamos a hacer la función de scan para una celda tipo RNN vainilla

In [7]:
def scan_V_RNN(elems, weights, h_0=None): # Forward propagation for RNNs
    h_t = h_0
    h = []
    for x in elems:
        h_t = forward_V_RNN([x, h_t], weights)
        h.append(h_t)
    return h, h_t

Vamos a probar inicializando una posible red RNN vainilla en un probable pornblema de PLN

In [8]:
np.random.seed(10)

emb = 128                       # Embedding
T = 256                         # Tamaño de secuencia de tokens
h_dim = 16                      # Estados ocultos

h_0 = np.zeros((h_dim, 1))      # Estado inicial

# Inicialización aleatoria de pesos y sesgos
Whh = np.random.standard_normal((h_dim, h_dim))
Wxh = np.random.standard_normal((h_dim, emb))
bh = np.random.standard_normal((h_dim, 1))

# Inicialización aleatoria de una secuencia de tokens (en embeddings)
X = np.random.standard_normal((T, emb, 1))

weights = [Whh, Wxh, bh]

In [9]:
# vanilla RNNs
tic = perf_counter()
h, h_T = scan_V_RNN(X, weights, h_0)
toc = perf_counter()
RNN_time=(toc-tic)*1000
print (f"Tomó {RNN_time:.2f}ms ejecutar el método de RNN vainilla.")

Tomó 2.47ms ejecutar el método de RNN vainilla.


**Desarrolla la función de scan para LSTM y prueba con la misma secuencia de entradas para una red LSTM**

In [10]:
# Función scan para LSTM

# INICIA CODIGO
def scan_LSTM(elems, weights, h_0=None, C_0=None):
    h_t = h_0
    C_t = C_0
    h = []
    C = []

    for x in elems:
        h_t, C_t = forward_LSTM([x, h_t, C_t], weights)
        h.append(h_t)
        C.append(C_t)

# TERMINA CODIGO
    return h, C, h_t, C_t

In [11]:
# Inicialización de variables

# INICIA CODIGO
np.random.seed(10)

emb = 128                       # Embedding
T = 256                         # Tamaño de secuencia de tokens
h_dim = 16                      # Estados ocultos

h_0 = np.zeros((h_dim, 1))
C_0 = np.zeros((h_dim, 1))


# Inicialización aleatoria de pesos y sesgos
Ui = np.random.standard_normal((h_dim, emb))
Wi = np.random.standard_normal((h_dim, h_dim))
Uf = np.random.standard_normal((h_dim, emb))
Wf = np.random.standard_normal((h_dim, h_dim))
Uo = np.random.standard_normal((h_dim, emb))
Wo = np.random.standard_normal((h_dim, h_dim))
U = np.random.standard_normal((h_dim, emb))
W = np.random.standard_normal((h_dim, h_dim))

# Inicialización aleatoria de una secuencia de tokens (en embeddings)
X = np.random.standard_normal((T, emb, 1))

weights = [Ui, Wi, Uf, Wf, Uo, Wo, U, W]

# TERMINA CODIGO

In [12]:
# Probando la función de scan

# INICIA CODIGO
tic = perf_counter()
h, C, h_T, C_T = scan_LSTM(X, weights, h_0, C_0)
toc = perf_counter()
LSTM_time = (toc - tic) * 1000
print(f"Tomó {LSTM_time:.2f}ms ejecutar el método de LSTM.")

# TERMINA CODIGO

Tomó 10.38ms ejecutar el método de LSTM.
